In [50]:
# Cell 1: Imports and Setup
import os
import pandas as pd

from langchain_ollama import ChatOllama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ConversationBufferWindowMemory
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent

# LLM - Ollama (runs locally)
llm = ChatOllama(model="llama3.2", temperature=0.3)

# Embeddings - HuggingFace (tiny, fast, no crashes)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

print("✅ Setup complete")

✅ Setup complete


In [51]:
# Cell 2: Mock Vectorstore (Member 1 will replace this with the real FAISS index)
# Using real World Cup facts so your chains give realistic answers
#vectorstore = FAISS.load_local("faiss_worldcup_index", embeddings, allow_dangerous_deserialization=True)
mock_docs = [
    Document(page_content="The 2014 FIFA World Cup was held in Brazil. The winner was Germany. The runner-up was Argentina. Third place went to Netherlands and fourth place to Brazil. A total of 171 goals were scored across 64 matches. 32 teams qualified. Total attendance was 3429873.", metadata={"year": 2014, "type": "tournament"}),
    Document(page_content="The 2010 FIFA World Cup was held in South Africa. The winner was Spain. The runner-up was Netherlands. Third place went to Germany and fourth place to Uruguay. A total of 145 goals were scored across 64 matches. 32 teams qualified.", metadata={"year": 2010, "type": "tournament"}),
    Document(page_content="The 1998 FIFA World Cup was held in France. The winner was France. The runner-up was Brazil. Third place went to Croatia. A total of 171 goals were scored across 64 matches. 32 teams qualified.", metadata={"year": 1998, "type": "tournament"}),
    Document(page_content="The 2002 FIFA World Cup was held in Korea/Japan. The winner was Brazil. The runner-up was Germany. Third place went to Turkey. 161 goals were scored across 64 matches.", metadata={"year": 2002, "type": "tournament"}),
    Document(page_content="World Cup statistics for Brazil: Played 109 matches. Record: 73 wins, 18 draws, 18 losses. Goals scored: 229. Goals conceded: 105. Win rate: 67.0%. Won the World Cup in: 1958, 1962, 1970, 1994, 2002. Participated in every World Cup tournament.", metadata={"type": "team_stats", "team": "Brazil"}),
    Document(page_content="World Cup statistics for Germany: Played 106 matches. Record: 66 wins, 20 draws, 20 losses. Goals scored: 224. Goals conceded: 121. Win rate: 62.3%. Won the World Cup in: 1954, 1974, 1990, 2014.", metadata={"type": "team_stats", "team": "Germany"}),
    Document(page_content="World Cup statistics for Argentina: Played 77 matches. Record: 42 wins, 14 draws, 21 losses. Goals scored: 131. Goals conceded: 84. Win rate: 54.5%. Won the World Cup in: 1978, 1986.", metadata={"type": "team_stats", "team": "Argentina"}),
    Document(page_content="World Cup statistics for France: Played 59 matches. Record: 34 wins, 11 draws, 14 losses. Goals scored: 120. Goals conceded: 71. Win rate: 57.6%. Won the World Cup in: 1998.", metadata={"type": "team_stats", "team": "France"}),
    Document(page_content="Head-to-head World Cup record between Brazil and Germany: Met 5 times. Brazil wins: 3, Germany wins: 1, Draws: 1. Key matches: 2014 Semi-final: Brazil 1-7 Germany; 2002 Final: Brazil 2-0 Germany.", metadata={"type": "h2h", "team1": "Brazil", "team2": "Germany"}),
    Document(page_content="Head-to-head World Cup record between Argentina and Germany: Met 7 times. Germany wins: 4, Argentina wins: 2, Draws: 1. Key matches: 2014 Final: Germany 1-0 Argentina; 2010 Quarter-final: Germany 4-0 Argentina.", metadata={"type": "h2h", "team1": "Argentina", "team2": "Germany"}),
    Document(page_content="Head-to-head World Cup record between Brazil and France: Met 3 times. France wins: 2, Brazil wins: 1. Key matches: 1998 Final: Brazil 0-3 France; 2006 Quarter-final: Brazil 0-1 France.", metadata={"type": "h2h", "team1": "Brazil", "team2": "France"}),
    Document(page_content="In the 2014 World Cup Final, Germany played against Argentina. Score: Germany 1-0 Argentina (after extra time). Played at Maracana in Rio De Janeiro. Attendance: 74738.", metadata={"year": 2014, "type": "match", "stage": "Final"}),
    Document(page_content="In the 2014 World Cup Semi-final, Brazil played against Germany. Score: Brazil 1-7 Germany. Played at Estadio Mineirao in Belo Horizonte. Attendance: 58141.", metadata={"year": 2014, "type": "match", "stage": "Semi-finals"}),
    Document(page_content="In the 1998 World Cup Final, Brazil played against France. Score: Brazil 0-3 France. Played at Stade de France in Saint-Denis. Attendance: 80000.", metadata={"year": 1998, "type": "match", "stage": "Final"}),
]

vectorstore = FAISS.from_documents(mock_docs, embeddings)
print(f"✅ Mock vectorstore ready with {len(mock_docs)} documents")

✅ Mock vectorstore ready with 14 documents


In [52]:
# Cell 3: Create a Retriever and test it

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# Test: does it find the right documents for a question?
results = retriever.invoke("Who won the 2014 World Cup?")

print(f"Search returned {len(results)} documents:\n")
for i, doc in enumerate(results):
    print(f"--- Result {i+1} ---")
    print(doc.page_content[:150])
    print()

Search returned 4 documents:

--- Result 1 ---
The 2014 FIFA World Cup was held in Brazil. The winner was Germany. The runner-up was Argentina. Third place went to Netherlands and fourth place to B

--- Result 2 ---
In the 2014 World Cup Final, Germany played against Argentina. Score: Germany 1-0 Argentina (after extra time). Played at Maracana in Rio De Janeiro. 

--- Result 3 ---
In the 2014 World Cup Semi-final, Brazil played against Germany. Score: Brazil 1-7 Germany. Played at Estadio Mineirao in Belo Horizonte. Attendance: 

--- Result 4 ---
The 2010 FIFA World Cup was held in South Africa. The winner was Spain. The runner-up was Netherlands. Third place went to Germany and fourth place to



In [53]:
# Cell 4: UPGRADED RAG Chain — Chain-of-Thought + Source Citations

QA_SYSTEM_PROMPT = """You are the World Cup AI Analyst — an expert football historian 
covering FIFA World Cup tournaments from 1930 to 2014.

Use ONLY the provided context to answer. Follow this process:

STEP 1 - THINK: Identify which pieces of context are relevant to the question.
STEP 2 - REASON: Connect the facts logically to form your answer.
STEP 3 - ANSWER: Give your final response with citations.

Format your response like this:

💭 Reasoning:
[1-2 sentences showing your thought process — which data you're using and why]

📋 Answer:
[Your clear, concise answer — lead with the direct answer, then supporting details]
[Use specific numbers: years, scores, goals, attendance]
[Keep it 3-5 sentences for simple questions, more for complex ones]

📎 Sources:
[List which data you used, e.g., "Tournament data: 2014 World Cup", "Team stats: Brazil"]

Rules:
- NEVER make up statistics. Only use what's in the context.
- If the context doesn't have enough info, say: "Based on available data..." and state what's missing.
- Be engaging — like a football commentator who loves the game.
- This data covers 1930-2014 only. Say so if asked about other years.

Context:
{context}
"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", QA_SYSTEM_PROMPT),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("✅ Upgraded RAG chain — chain-of-thought + citations")

✅ Upgraded RAG chain — chain-of-thought + citations


In [54]:
# Cell 5: Add Memory + Ask Function

memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history",
    return_messages=True
)

def ask_worldcup(question: str) -> str:
    """Ask a question about World Cup history with memory."""
    chat_history = memory.load_memory_variables({})["chat_history"]
    
    response = rag_chain.invoke({
        "input": question,
        "chat_history": chat_history
    })
    
    memory.save_context(
        {"input": question},
        {"output": response["answer"]}
    )
    
    return response["answer"]

print("✅ Memory + ask function ready")

✅ Memory + ask function ready


In [55]:
# Cell 6: Test the RAG Chain

print("Q1: Who won the 2014 World Cup?")
print("A:", ask_worldcup("Who won the 2014 World Cup?"))
print()
print("=" * 50)
print()
print("Q2: How many goals were scored in that tournament?")
print("A:", ask_worldcup("How many goals were scored in that tournament?"))

Q1: Who won the 2014 World Cup?
A: 🏆
Germany won the 2014 FIFA World Cup, defeating Argentina 1-0 in the final after extra time, played at Maracana in Rio De Janeiro with an attendance of 74738. This marked Germany's fourth World Cup title.

📋 Sources:
Tournament data: 2014 World Cup


Q2: How many goals were scored in that tournament?
A: A total of 171 goals were scored across 64 matches during the 1998 FIFA World Cup.

📋 Sources:
Tournament data: 1998 World Cup


In [56]:
# Cell 7: Prediction chain built

PREDICTION_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an elite World Cup match analyst. Using historical data provided,
produce a data-driven match preview using the STAR analysis framework.

Use the following context:
{context}

Structure your response EXACTLY like this:

🏟️ MATCH ANALYSIS: [Team1] vs [Team2]

📌 SITUATION
- [Team1]: [W]-[D]-[L] record, [win rate]%, [titles] title(s) in [years]
- [Team2]: [W]-[D]-[L] record, [win rate]%, [titles] title(s) in [years]
- Head-to-head: [X] meetings — [summary of who leads]
- Last meeting: [match and score]

🎯 TASK
- [Team1] must: [what data says they need to do — e.g., "overcome a 1-4 H2H deficit"]
- [Team2] must: [what data says they need to do]

⚡ ACTION
- [Team1] strength: [backed by specific stat from context]
- [Team2] strength: [backed by specific stat from context]
- Deciding factor: [one key stat that tips the balance]

🏆 RESULT
- Prediction: [Team1] [X] - [Y] [Team2]
- Winner: [team]
- Reasoning: [2-3 sentences citing specific numbers from the data above]
- Confidence: [Low/Medium/High] — [why]

📎 Data Used: [List sources — e.g., "Brazil team stats", "Brazil vs Germany H2H record", "2014 Semi-final match data"]

⚠️ This analysis uses historical FIFA World Cup data (1930-2014) for educational purposes only.
It does not reflect current form, rosters, or real-world conditions."""),
    ("human", "Predict the match outcome: {team1} vs {team2}")
])

prediction_chain = create_stuff_documents_chain(llm, PREDICTION_PROMPT)

print("✅ Prediction chain built")

✅ Prediction chain built


In [57]:
# Cell 8: Predict Match Function (multi-query retrieval)

def predict_match(team1: str, team2: str) -> str:
    """Generate a match prediction between two teams."""
    
    # Search 1: Head-to-head history
    h2h_docs = vectorstore.similarity_search(
        f"{team1} vs {team2} head to head World Cup", k=3
    )
    
    # Search 2: Team 1 stats
    team1_docs = vectorstore.similarity_search(
        f"{team1} World Cup statistics record", k=2
    )
    
    # Search 3: Team 2 stats
    team2_docs = vectorstore.similarity_search(
        f"{team2} World Cup statistics record", k=2
    )
    
    # Combine and remove duplicates
    all_docs = h2h_docs + team1_docs + team2_docs
    seen = set()
    unique_docs = []
    for doc in all_docs:
        if doc.page_content not in seen:
            seen.add(doc.page_content)
            unique_docs.append(doc)
    
    print(f"   Retrieved {len(unique_docs)} unique documents for prediction")
    
    # Run prediction chain
    response = prediction_chain.invoke({
        "context": unique_docs,
        "team1": team1,
        "team2": team2
    })
    
    return response

print("✅ Predict function ready")

✅ Predict function ready


In [58]:
# Cell 9: Test Prediction

print("MATCH PREDICTION: Brazil vs Germany")
print(predict_match("Brazil", "Germany"))

MATCH PREDICTION: Brazil vs Germany
   Retrieved 5 unique documents for prediction
🏟️ MATCH ANALYSIS: Brazil vs Germany

📌 SITUATION
- Brazil: 73W-18D-18L, 67.0% win rate, 5 World Cup titles in 1958, 1962, 1970, 1994, and 2002.
- Germany: 66W-20D-20L, 62.3% win rate, 4 World Cup titles in 1954, 1974, 1990, and 2014.
- Head-to-head: 5 meetings — Brazil leads 3-1.
- Last meeting: 2014 Semi-final, Germany won 7-1.

🎯 TASK
- Brazil must overcome a significant H2H deficit (4-1) and find a way to contain Germany's high-scoring attack.
- Germany must capitalize on their experience in big games and exploit Brazil's defensive vulnerabilities.

⚡ ACTION
- Brazil strength: Brazil has a strong record against weaker opponents, with 52W-8D-6L against teams ranked outside the top 10.
- Germany strength: Germany excels in high-pressure situations, with a 55W-15D-12L record in knockout stage matches.

Deciding factor: Germany's ability to score multiple goals in big games (e.g., 7-1 vs Brazil).

🏆 RESU

In [59]:
# Cell 10: Define Agent Tools

@tool
def dataset_discovery_tool(query: str) -> str:
    """Discover what data is available in the World Cup dataset.
    Use this when the user asks what data you have, what years are covered,
    or what teams/tournaments are in the dataset."""
    return (
        "The dataset covers 20 FIFA World Cup tournaments from 1930 to 2014.\n"
        "Total matches: 852\n"
        "Total teams: 83\n"
        "Data includes: tournament summaries, match-level results, team statistics, "
        "head-to-head records, and player participation records.\n"
        "Teams include: Brazil, Germany, Argentina, France, Italy, Spain, "
        "England, Netherlands, Uruguay, and 74 more."
    )

@tool
def data_ingestion_tool(team_name: str) -> str:
    """Look up raw statistical data for a specific team from the dataset.
    Returns match counts, goals, and tournament participation.
    Use this when the user asks for a specific team's stats or record."""
    docs = vectorstore.similarity_search(f"{team_name} World Cup statistics record", k=2)
    if docs:
        return docs[0].page_content
    return f"No data found for '{team_name}'. Check the team name spelling."

@tool
def retrieval_or_filter_tool(question: str) -> str:
    """Search the World Cup knowledge base to answer factual questions about
    World Cup history, tournament results, match outcomes, and team records.
    Use this for any question about World Cup facts and history."""
    docs = vectorstore.similarity_search(question, k=6)
    return "\n\n".join([d.page_content for d in docs])

@tool
def reasoning_or_aggregation_tool(matchup: str) -> str:
    """Generate a match prediction between two teams. Input should be in
    the format 'Team1 vs Team2', e.g., 'Brazil vs Germany'.
    Analyzes head-to-head records and team stats to predict outcomes."""
    parts = matchup.split(" vs ")
    if len(parts) != 2:
        return "Please provide the matchup in format: 'Team1 vs Team2'"
    return predict_match(parts[0].strip(), parts[1].strip())

@tool
def report_generation_tool(topic: str) -> str:
    """Generate a structured report or summary about a World Cup topic.
    Good for questions like 'summarize the 1998 World Cup' or
    'give me a report on Brazil's World Cup history'."""
    docs = vectorstore.similarity_search(topic, k=8)
    context = "\n\n".join([d.page_content for d in docs])
    
    report_prompt = ChatPromptTemplate.from_messages([
        ("system", "Generate a well-structured report on the given topic using the context. "
         "Include relevant statistics, match results, and historical context. "
         "Note this covers World Cups 1930-2014.\n\nContext:\n{context}"),
        ("human", "{topic}")
    ])
    chain = report_prompt | llm
    return chain.invoke({"context": context, "topic": topic}).content

tools = [
    dataset_discovery_tool,
    data_ingestion_tool,
    retrieval_or_filter_tool,
    reasoning_or_aggregation_tool,
    report_generation_tool
]

print(f"✅ {len(tools)} tools defined")

✅ 5 tools defined


In [60]:
# Cell 10: Define Agent Tools (6 tools + user preference memory)

# Memory for user preferences (favorite team, etc.)
user_preferences = {}

@tool
def dataset_discovery_tool(query: str) -> str:
    """Discover what data is available in the World Cup dataset.
    Use this when the user asks what data you have, what years are covered,
    or what teams/tournaments are in the dataset."""
    return (
        "The dataset covers 20 FIFA World Cup tournaments from 1930 to 2014.\n"
        "Total matches: 852\n"
        "Total teams: 83\n"
        "Data includes: tournament summaries, match-level results, team statistics, "
        "head-to-head records, and player participation records.\n"
        "Teams include: Brazil, Germany, Argentina, France, Italy, Spain, "
        "England, Netherlands, Uruguay, and 74 more."
    )

@tool
def data_ingestion_tool(team_name: str) -> str:
    """Look up raw statistical data for a specific team from the dataset.
    Returns match counts, goals, and tournament participation.
    Use this when the user asks for a specific team's stats or record."""
    docs = vectorstore.similarity_search(f"{team_name} World Cup statistics record", k=2)
    if docs:
        # Check if user is asking about their favorite team
        if team_name.lower() in [v.lower() for v in user_preferences.values()]:
            prefix = f"Here are the stats for your favorite team, {team_name}!\n\n"
        else:
            prefix = ""
        return prefix + docs[0].page_content
    return f"No data found for '{team_name}'. Check the team name spelling."

@tool
def retrieval_or_filter_tool(question: str) -> str:
    """Search the World Cup knowledge base to answer factual questions about
    World Cup history, tournament results, match outcomes, and team records.
    Use this for any question about World Cup facts and history."""
    docs = vectorstore.similarity_search(question, k=6)
    return "\n\n".join([d.page_content for d in docs])

@tool
def reasoning_or_aggregation_tool(matchup: str) -> str:
    """Generate a match prediction between two teams. Input should be in
    the format 'Team1 vs Team2', e.g., 'Brazil vs Germany'.
    Analyzes head-to-head records and team stats to predict outcomes."""
    parts = matchup.split(" vs ")
    if len(parts) != 2:
        return "Please provide the matchup in format: 'Team1 vs Team2'"
    return predict_match(parts[0].strip(), parts[1].strip())

@tool
def report_generation_tool(topic: str) -> str:
    """Generate a structured report or summary about a World Cup topic.
    Good for questions like 'summarize the 1998 World Cup' or
    'give me a report on Brazil's World Cup history'."""
    docs = vectorstore.similarity_search(topic, k=8)
    context = "\n\n".join([d.page_content for d in docs])
    
    report_prompt = ChatPromptTemplate.from_messages([
        ("system", "Generate a well-structured report on the given topic using the context. "
         "Include relevant statistics, match results, and historical context. "
         "Note this covers World Cups 1930-2014.\n\nContext:\n{context}"),
        ("human", "{topic}")
    ])
    chain = report_prompt | llm
    return chain.invoke({"context": context, "topic": topic}).content

@tool
def comparison_tool(matchup: str) -> str:
    """Compare two teams side-by-side with detailed statistics.
    Input format: 'Team1 vs Team2', e.g., 'Brazil vs Argentina'.
    Use this when the user asks to compare teams, asks which team is better,
    or wants a side-by-side analysis."""
    parts = matchup.split(" vs ")
    if len(parts) != 2:
        return "Please provide in format: 'Team1 vs Team2'"
    
    team1, team2 = parts[0].strip(), parts[1].strip()
    
    # Retrieve stats for both teams
    t1_docs = vectorstore.similarity_search(f"{team1} World Cup statistics", k=2)
    t2_docs = vectorstore.similarity_search(f"{team2} World Cup statistics", k=2)
    h2h_docs = vectorstore.similarity_search(f"{team1} vs {team2} head to head", k=2)
    
    all_context = t1_docs + t2_docs + h2h_docs
    context = "\n\n".join([d.page_content for d in all_context])
    
    compare_prompt = ChatPromptTemplate.from_messages([
        ("system", """Compare these two World Cup teams side-by-side using the data provided.

Format your response as:

📊 TEAM COMPARISON: [Team1] vs [Team2]

| Category | [Team1] | [Team2] | Edge |
|----------|---------|---------|------|
| Matches Played | [X] | [Y] | [who leads] |
| Win Rate | [X%] | [Y%] | [who leads] |
| Goals Scored | [X] | [Y] | [who leads] |
| Goals Conceded | [X] | [Y] | [who leads] |
| World Cup Titles | [X] | [Y] | [who leads] |

⚔️ HEAD-TO-HEAD: [summary]

🏆 VERDICT: [Which team has the statistical edge overall and why — 2-3 sentences]

📎 Data Used: [list sources]

Context:
{context}"""),
        ("human", "Compare {team1} vs {team2}")
    ])
    
    chain = compare_prompt | llm
    return chain.invoke({"context": context, "team1": team1, "team2": team2}).content

tools = [
    dataset_discovery_tool,
    data_ingestion_tool,
    retrieval_or_filter_tool,
    reasoning_or_aggregation_tool,
    report_generation_tool,
    comparison_tool
]

print(f"✅ {len(tools)} tools defined (including comparison tool)")

✅ 6 tools defined (including comparison tool)


In [61]:
# Cell 11: Build the Agent (with preference memory + 6 tools)

AGENT_SYSTEM_PROMPT = """You are the World Cup AI Analyst, an expert chatbot on FIFA World Cup 
history from 1930 to 2014.

You have 6 tools:
- dataset_discovery_tool: Find what data is available
- data_ingestion_tool: Look up raw stats for a specific team
- retrieval_or_filter_tool: Search the knowledge base for factual answers
- reasoning_or_aggregation_tool: Predict match outcomes (format: 'Team1 vs Team2')
- report_generation_tool: Generate structured reports on topics
- comparison_tool: Compare two teams side-by-side (format: 'Team1 vs Team2')

TOOL SELECTION GUIDE:
- "Who won..." / "How many..." / "When did..." → retrieval_or_filter_tool
- "Predict..." / "What would happen if..." → reasoning_or_aggregation_tool
- "Compare..." / "Which is better..." / "X vs Y stats" → comparison_tool
- "Tell me about [team]..." / "[team] stats" → data_ingestion_tool
- "Summarize..." / "Report on..." → report_generation_tool
- "What data..." / "What teams..." → dataset_discovery_tool

USER PREFERENCE HANDLING:
- If the user says "my favorite team is X" or "I support X", remember it and use it 
  to personalize future responses.
- If the user has a favorite team, relate answers back to their team when relevant.

Guidelines:
- Always use tools to ground responses in real data.
- Be enthusiastic and conversational about football.
- Cite specific stats, scores, and years.
- For predictions, note they are educational and based on historical data only.
- If data is insufficient, say so honestly.
"""

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", AGENT_SYSTEM_PROMPT),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent = create_tool_calling_agent(llm, tools, agent_prompt)

agent_memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history",
    return_messages=True
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=agent_memory,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

print("✅  Agent ready ")

✅  Agent ready 


In [62]:
# Cell 12: Test the Agent

print("TEST 1: Factual question")
response = agent_executor.invoke({"input": "Who won the 2014 World Cup?"})
print("\nFinal Answer:", response["output"])

TEST 1: Factual question


> Entering new AgentExecutor chain...

Invoking: `retrieval_or_filter_tool` with `{'question': 'Who won the 2014 World Cup?'}`


The 2014 FIFA World Cup was held in Brazil. The winner was Germany. The runner-up was Argentina. Third place went to Netherlands and fourth place to Brazil. A total of 171 goals were scored across 64 matches. 32 teams qualified. Total attendance was 3429873.

In the 2014 World Cup Final, Germany played against Argentina. Score: Germany 1-0 Argentina (after extra time). Played at Maracana in Rio De Janeiro. Attendance: 74738.

In the 2014 World Cup Semi-final, Brazil played against Germany. Score: Brazil 1-7 Germany. Played at Estadio Mineirao in Belo Horizonte. Attendance: 58141.

The 2010 FIFA World Cup was held in South Africa. The winner was Spain. The runner-up was Netherlands. Third place went to Germany and fourth place to Uruguay. A total of 145 goals were scored across 64 matches. 32 teams qualified.

Head-to-head World Cup 

In [63]:
# Cell 13: Test prediction and memory

print("TEST 2: Match Prediction")
response = agent_executor.invoke({"input": "Predict Brazil vs Germany"})
print("\nFinal Answer:", response["output"])

TEST 2: Match Prediction


> Entering new AgentExecutor chain...

Invoking: `reasoning_or_aggregation_tool` with `{'matchup': 'Brazil vs Germany'}`


   Retrieved 5 unique documents for prediction
🏟️ MATCH ANALYSIS: Brazil vs Germany

📌 SITUATION
- Brazil: 73W-18D-18L, 67.0% win rate, 5 World Cup titles in 1958, 1962, 1970, 1994, and 2002.
- Germany: 66W-20D-20L, 62.3% win rate, 4 World Cup titles in 1954, 1974, 1990, and 2014.
- Head-to-head: 5 meetings — Brazil leads 3-1 (Draws: 1).
- Last meeting: 2014 Semi-final, Germany won 7-1.

🎯 TASK
- Brazil must overcome a significant H2H deficit in goals scored (Germany +24, Brazil +13) and rely on their high win rate to outperform Germany.
- Germany must exploit Brazil's defensive vulnerabilities (conceded 105 goals in 109 matches) and capitalize on their experience in big games.

⚡ ACTION
- Brazil strength: High win rate in World Cup matches (67.0%).
- Germany strength: Strong record in knockout stages of the World Cup (66W-20D-20L).
- Dec

In [64]:
# Cell 14: Test memory (follow-up question)

print("TEST 3: Memory test")
response = agent_executor.invoke({"input": "How many goals were scored in that tournament we discussed?"})
print("\nFinal Answer:", response["output"])

TEST 3: Memory test


> Entering new AgentExecutor chain...

Invoking: `data_ingestion_tool` with `{'team_name': 'World Cup 2014'}`


World Cup statistics for Germany: Played 106 matches. Record: 66 wins, 20 draws, 20 losses. Goals scored: 224. Goals conceded: 121. Win rate: 62.3%. Won the World Cup in: 1954, 1974, 1990, 2014.According to the data ingestion tool, a total of 244 goals were scored during the 2014 World Cup. Germany scored 224 goals and conceded 121 goals.

> Finished chain.

Final Answer: According to the data ingestion tool, a total of 244 goals were scored during the 2014 World Cup. Germany scored 224 goals and conceded 121 goals.


In [65]:
# Cell 15: Test report generation

print("TEST 4: Report generation")
response = agent_executor.invoke({"input": "Give me a report on the 1998 World Cup"})
print("\nFinal Answer:", response["output"])

TEST 4: Report generation


> Entering new AgentExecutor chain...

Invoking: `report_generation_tool` with `{'topic': 'the 1998 World Cup'}`


**The 1998 FIFA World Cup: A Historic Tournament**

Introduction:

The 1998 FIFA World Cup was the 15th edition of the FIFA World Cup, held in France from June 10 to July 12, 1998. The tournament featured 32 teams and was won by France, marking their first World Cup title.

**Tournament Overview**

* Date: June 10 - July 12, 1998
* Host Country: France
* Number of Teams: 32
* Matches Played: 64
* Total Goals Scored: 171

**Group Stage Results**

The 1998 World Cup was divided into eight groups of four teams each. The top two teams from each group advanced to the knockout stage.

**Knockout Stage Results**

* Round of 16:
	+ France defeated Paraguay 1-0
	+ Brazil defeated Norway 3-2 (after a penalty shootout)
	+ Netherlands defeated South Korea 4-1
	+ Germany defeated Mexico 1-0
* Quarterfinals:
	+ France defeated Paraguay 1-0
	+ Brazil defeated 